In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Load the MNIST dataset, shuffled and supervised (returning tuple (img, label)
# instead of dictionary {'image': img, 'label': label})
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files = True,
    as_supervised = True,
    with_info = True
)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  50%|█████     | 2/4 [00:00<00:00,  3.76 url/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:00<00:00,  7.06 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  4.20 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  3.08 url/s]


Dataset mnist downloaded and prepared to C:\Users\itsno\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [3]:
# Building a training set pipeline

def normalize_img(image, label):
  '''Normalizes images from 'uint-8' to 'float32'''
  return tf.cast(image, tf.float32) / 255.0, label

# Normalize our training set, with a dynamic number of parallel calls
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Cache our training set for better performance (done before random transforms)
ds_train = ds_train.cache()

# Shuffle training set, with buffer set to full dataset size for max randomness
# Can be lowered if large dataset cannot fit into memory
ds_train = ds_train.shuffle(buffer_size = ds_info.splits['train'].num_examples)

# Self-explanatory, batches up elements to get unique batches at each epoch
ds_train = ds_train.batch(batch_size = 128)

# Overlaps preprocessing and model execution for performance
# Good practice to end off pipelines with this!
ds_train = ds_train.prefetch(buffer_size = tf.data.AUTOTUNE)

In [4]:
# Building a test set pipeline, same as training

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(buffer_size = tf.data.AUTOTUNE)

In [5]:
# Create the model

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
)

In [6]:
# Train the model

model.fit(
    ds_train,
    epochs = 10,
    validation_data = ds_test
)

Epoch 1/10
469/469 [==============================] - 4s 3ms/step - loss: 0.3614 - sparse_categorical_accuracy: 0.8995 - val_loss: 0.2003 - val_sparse_categorical_accuracy: 0.9425
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 0.1681 - sparse_categorical_accuracy: 0.9513 - val_loss: 0.1404 - val_sparse_categorical_accuracy: 0.9582
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.1225 - sparse_categorical_accuracy: 0.9655 - val_loss: 0.1177 - val_sparse_categorical_accuracy: 0.9644
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0949 - sparse_categorical_accuracy: 0.9725 - val_loss: 0.1007 - val_sparse_categorical_accuracy: 0.9698
Epoch 5/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0773 - sparse_categorical_accuracy: 0.9772 - val_loss: 0.0930 - val_sparse_categorical_accuracy: 0.9721
Epoch 6/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0641 - sparse_categorica

In [9]:
# Save model

model.save("model.keras")